# Setup for local

In [14]:
%cd ../
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 
from datetime import datetime, timedelta
import os
import requests
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

/home/harshapk/Projects
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from css_proj.preprocess.consts import COVID_LIES_PATH
df = pd.read_csv(COVID_LIES_PATH)
df

,misconception_id,misconception,tweet_id,label
0,3,Coronavirus is genetically engineered.,1233965490948591616,na
1,30,Blowing conch shells destroys coronavirus pote...,1233907923765559296,na
2,57,Swans and dolphins swimming in Venice canals f...,1233911842910720000,na
3,22,Cocaine cures coronavirus.,1233947734094290944,na
4,32,Observing janata curfew will result in the red...,1233937085297332224,na
...,...,...,...,...
6586,50,It is safe for individuals infected with COVID...,1233949547715584000,na
6587,19,Vitamin C cures coronavirus.,1233966049097285632,na
6588,22,Cocaine cures coronavirus.,1233965616316334080,na
6589,61,Lions were freed to keep people off the street...,1233906211310825472,na


In [3]:
API_KEY = "tpzOojAJ6KxsoNTnansWU6665"
API_SECRET = "G7aIqSMfrELwzptV4G9pgOBAm3tr1mvIML7Hl8kBivDvXwUkyc"
ACCESS_KEY = "987583707346358272-y5cPBhuOpHStAdy2lmYGJEfZHNAqvoZ"
ACCESS_SECRET = "JN151eja8TFbGYTyIUJD3KmDwbN2vjJ2yGPbXhWkvpe4c"

In [4]:
import tweepy
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
api = tweepy.API(auth)

In [5]:
# Example
tweet = api.get_status("1233949547715584000")

In [6]:
print(tweet.text)

One Spokane County resident is displaying symptoms of COVID-19 and has been tested for the respiratory disease, whi… https://t.co/gZd91v7M6Q


In [7]:
print(tweet.user.screen_name)
print(tweet.user.description)
print(tweet.user.location)
print(tweet.user.friends_count) # Following
print(tweet.user.followers_count)
print(tweet.user.verified)
print(tweet.user.statuses_count)
print(tweet.user.favourites_count) # Total no. of likes
print(tweet.user.listed_count)

SpokesmanReview
The Spokesman-Review is the Spokane region's only daily newspaper, covering news, sports, and life in Eastern Washington and North Idaho since 1883.
Spokane, WA
1485
61280
False
69263
1544
944


In [8]:
print(tweet.retweet_count,tweet.favorite_count,tweet.created_at)
print(tweet.lang, tweet.is_quote_status, tweet.source)

13 9 2020-03-01 02:57:52+00:00
en False TweetDeck


In [10]:
followers = tweet.user.followers()

We got some useful user based and tweet based features. Collect them into a dataframe

In [11]:
features = ['tweet_id', 'tweet_text', 'retweets', 'likes', 'created', 'lang', 'device', 
           'user_id', 'user_name', 'user_handle', 'user_location',
            'user_following', 'user_followers', 'user_verified', 
            'user_total_tweets', 'user_total_likes', 'user_total_lists',
            'misconception_id', 'label'
           ]

In [15]:
dataset = pd.DataFrame(columns=features)
for i in tqdm(range(len(df))):
    entry = dict()
    entry['tweet_id'] = df.iloc[i]['tweet_id']
    try:
        tweet = api.get_status(str(entry['tweet_id']))
        entry['tweet_text'] = tweet.text
        entry['retweets'] = tweet.retweet_count
        entry['likes'] = tweet.favorite_count
        entry['created'] = tweet.created_at
        entry['lang'] = tweet.lang
        entry['device'] = tweet.source
        user = tweet.user
        entry['user_id'] = user.id
        entry['user_name'] = user.name
        entry['user_handle'] = user.screen_name
        entry['user_location'] = user.location
        entry['user_following'] = user.friends_count
        entry['user_followers'] = user.followers_count
        entry['user_verified'] = user.verified
        entry['user_total_tweets'] = user.statuses_count
        entry['user_total_likes'] = user.favourites_count
        entry['user_total_lists'] = user.listed_count
        entry['misconception_id'] = df.iloc[i]['misconception_id']
        entry['label'] = df.iloc[i]['label']
        dataset = dataset.append(entry, ignore_index=True)
    except:
        print("Error for tweet id:", entry['tweet_id'])

  0%|          | 1/6591 [00:00<28:42,  3.83it/s]

Error for tweet id: 1233965490948591616


/tmp/ipykernel_26906/365348384.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(entry, ignore_index=True)
  0%|          | 2/6591 [00:00<42:33,  2.58it/s]/tmp/ipykernel_26906/365348384.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(entry, ignore_index=True)
  0%|          | 3/6591 [00:01<38:03,  2.89it/s]/tmp/ipykernel_26906/365348384.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(entry, ignore_index=True)
  0%|          | 5/6591 [00:01<32:07,  3.42it/s]

Error for tweet id: 1233937085297332224


  0%|          | 6/6591 [00:01<30:40,  3.58it/s]

...

Error for tweet id: 1233965616316334080


100%|█████████▉| 6590/6591 [27:43<00:00,  4.43it/s]

Error for tweet id: 1233906211310825472


100%|██████████| 6591/6591 [27:43<00:00,  3.96it/s]

Error for tweet id: 1233983733906075648


In [16]:
dataset.head()

,tweet_id,tweet_text,retweets,likes,created,lang,device,user_id,user_name,user_handle,user_location,user_following,user_followers,user_verified,user_total_tweets,user_total_likes,user_total_lists,misconception_id,label
0,1233907923765559296,Getting coronavirus and then coughing on peopl...,1,0,2020-03-01 00:12:28+00:00,en,Twitter Web App,1205736570147524609,Cлава Сталинy 🇺🇲🤝🇨🇳,Ancommissar,Ministry for State Security,199,211,False,1464,25645,2,30,na
1,1233911842910720000,@Barioth_a_bot そう…ですね…私は常時深海に居ります故…ふわふわ、という感触で...,0,0,2020-03-01 00:28:02+00:00,ja,海底遺跡,369422192,ナバルデウス,Navaldeus_bot,海底遺跡,929,1093,False,71289,1,38,57,na
2,1233947734094290944,RT @TabascoHOY: #Coronavirus☣\n➡Los infectados...,1,0,2020-03-01 02:50:40+00:00,es,Twitter for Android,162886483,Fx,ELECTROGALACTIC,México,1534,525,False,95919,140760,3,22,na
3,1233952635725778944,Branco já nasce sem noção ou enfia ela no cu d...,1,10,2020-03-01 03:10:08+00:00,pt,Twitter for iPhone,213424175,vic,vvegansexual,,533,488,False,32576,48600,0,3,na
4,1233920123183828992,"“Most people (not elderly, no underlying healt...",0,0,2020-03-01 01:00:57+00:00,en,Twitter Web Client,282696635,Eileen May,suleskerry,,2045,2846,False,197924,79634,13,50,na


In [21]:
import pickle
path = "./data/data_processed/covid_lies_dataset.pkl"
with open(path, 'wb') as f:
    pickle.dump(dataset, f)

In [20]:
%cd CSS_Proj

/home/harshapk/Projects/CSS_Proj


In [23]:
len(dataset)

1256